In [1]:
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit

from qiskit.primitives import BackendEstimator, Estimator, Sampler
from qiskit import Aer

from squlearn.util.optree import *
from squlearn.util.optree.optree import *
from squlearn.util.optree.optree_derivative import *
from squlearn.util.optree.optree_evaluate import *

# circuit = random_circuit(4, 2, seed=12)
# circuit.draw()

pqc_param = ParameterVector("θ", 4)

circuit = QuantumCircuit(4)
circuit.h([0, 1, 2, 3])
circuit.u(0.5, 0.5, 0.5, [0, 1, 2, 3])
circuit.cx(0, 1)
circuit.cx(2, 3)
circuit.cx(1, 2)

circuit2 = circuit.copy()

circuit.ry(pqc_param[0], 0)
circuit.ry(pqc_param[1], 1)
circuit.ry(pqc_param[2], 2)
circuit.ry(pqc_param[3], 3)
circuit.draw()

circuit2.rx(pqc_param[0], 0)
circuit2.rx(pqc_param[1], 1)
circuit2.rx(pqc_param[2], 2)
circuit2.rx(pqc_param[3], 3)

In [2]:
param1 = {pqc_param[0]: 0.5, pqc_param[1]: 0.5, pqc_param[2]: 0.5, pqc_param[3]: 0.5}
param2 = {pqc_param[0]: 0.15, pqc_param[1]: 0.15, pqc_param[2]: 0.85, pqc_param[3]: 0.95}

In [3]:
op_param = ParameterVector("p", 3)
op = SparsePauliOp(["XXXX", "YYYY", "IIYY"], [op_param[0], op_param[1], op_param[2]])
op_dict1 = {op_param[0]: 0.5, op_param[1]: 0.5, op_param[2]: 0.5}
op_dict2 = {op_param[0]: 0.25, op_param[1]: 0.25, op_param[2]: 0.25}

In [4]:
op.num_qubits
op_clean = SparsePauliOp(["ZZZZ", "ZZZI", "IIZZ"], [op_param[0], op_param[1], op_param[2]])
op_clean_param = ParameterVector("p", 3)
op_clean_dict1 = {op_clean_param[0]: 0.5, op_clean_param[1]: 0.2, op_clean_param[2]: 0.3}
op_clean_dict2 = {op_clean_param[0]: 0.12, op_clean_param[1]: 0.8, op_clean_param[2]: -0.3}

In [5]:
op_clean2 = SparsePauliOp(["IIZZ", "ZZII"], [0.1, 0.2])

In [6]:
opdic1 = copy.copy(op_dict1)
opdic1.update(op_clean_dict1)

opdic2 = copy.copy(op_dict2)
opdic2.update(op_clean_dict2)

In [7]:
test = transform_operator_to_zbasis(op)
print(test)

(1.0*
     ┌───┐┌─┐         
q_0: ┤ H ├┤M├─────────
     ├───┤└╥┘┌─┐      
q_1: ┤ H ├─╫─┤M├──────
     ├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├─╫──╫─┤M├───
     ├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├─╫──╫──╫─┤M├
     └───┘ ║  ║  ║ └╥┘
c: 4/══════╩══╩══╩══╩═
           0  1  2  3 

 with observable 
SparsePauliOp(['ZZZZ'],
              coeffs=[ParameterExpression(1.0*p[0])])
 + 1.0*
     ┌─────┐┌───┐┌─┐         
q_0: ┤ Sdg ├┤ H ├┤M├─────────
     ├─────┤├───┤└╥┘┌─┐      
q_1: ┤ Sdg ├┤ H ├─╫─┤M├──────
     ├─────┤├───┤ ║ └╥┘┌─┐   
q_2: ┤ Sdg ├┤ H ├─╫──╫─┤M├───
     ├─────┤├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ Sdg ├┤ H ├─╫──╫──╫─┤M├
     └─────┘└───┘ ║  ║  ║ └╥┘
c: 4/═════════════╩══╩══╩══╩═
                  0  1  2  3 

 with observable 
SparsePauliOp(['ZZZZ', 'IIZZ'],
              coeffs=[ParameterExpression(1.0*p[1]), ParameterExpression(1.0*p[2])])
)


In [8]:
tree = OpTreeLeafExpectationValue(circuit, op)
print(tree)


     ┌───┐┌────────────────┐     ┌──────────┐            
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ Ry(θ[0]) ├────────────
     ├───┤├────────────────┤┌─┴─┐└──────────┘┌──────────┐
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────■──────┤ Ry(θ[1]) ├
     ├───┤├────────────────┤└───┘   ┌─┴─┐    ├──────────┤
q_2: ┤ H ├┤ U(0.5,0.5,0.5) ├──■─────┤ X ├────┤ Ry(θ[2]) ├
     ├───┤├────────────────┤┌─┴─┐┌──┴───┴───┐└──────────┘
q_3: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├┤ Ry(θ[3]) ├────────────
     └───┘└────────────────┘└───┘└──────────┘            

 with observable 
SparsePauliOp(['XXXX', 'YYYY', 'IIYY'],
              coeffs=[ParameterExpression(1.0*p[0]), ParameterExpression(1.0*p[1]),
 ParameterExpression(1.0*p[2])])



In [9]:
adjust_tree = transform_tree_to_zbasis(tree, abelian_grouping=False)
print(adjust_tree)

(1.0*
     ┌───┐┌────────────────┐     ┌──────────┐   ┌───┐    ┌─┐           
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ Ry(θ[0]) ├───┤ H ├────┤M├───────────
     ├───┤├────────────────┤┌─┴─┐└──────────┘┌──┴───┴───┐└╥┘┌───┐┌─┐   
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────■──────┤ Ry(θ[1]) ├─╫─┤ H ├┤M├───
     ├───┤├────────────────┤└───┘   ┌─┴─┐    ├──────────┤ ║ ├───┤└╥┘┌─┐
q_2: ┤ H ├┤ U(0.5,0.5,0.5) ├──■─────┤ X ├────┤ Ry(θ[2]) ├─╫─┤ H ├─╫─┤M├
     ├───┤├────────────────┤┌─┴─┐┌──┴───┴───┐└──┬───┬───┘ ║ └┬─┬┘ ║ └╥┘
q_3: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├┤ Ry(θ[3]) ├───┤ H ├─────╫──┤M├──╫──╫─
     └───┘└────────────────┘└───┘└──────────┘   └───┘     ║  └╥┘  ║  ║ 
c: 4/═════════════════════════════════════════════════════╩═══╩═══╩══╩═
                                                          0   3   1  2 

 with observable 
SparsePauliOp(['ZZZZ'],
              coeffs=[ParameterExpression(1.0*p[0])])
 + 1.0*
     ┌───┐┌────────────────┐     ┌──────────┐  ┌─────┐    ┌───┐ ┌─┐           
q_0: ┤ H ├┤ U(0.5,

In [10]:
evaluate_estimator(
    OpTreeNodeList([circuit, circuit2]),
    OpTreeNodeList([op, op_clean, op_clean2]),
    [param1, param2],
    [opdic1, opdic2],
    Estimator(),
    dictionaries_combined=False,
)

pre-processing 0.0029993057250976562
Number of circuits in the estimator: 24
Estimator run time 0.03299832344055176
post processing 0.0


array([[[[-0.26725525, -0.06118615,  0.06180155],
         [-0.25791181,  0.0742432 , -0.00303811]],

        [[-0.13362763, -0.03059308,  0.06180155],
         [-0.1289559 ,  0.0371216 , -0.00303811]]],


       [[[-0.17570876, -0.22534089,  0.07574052],
         [-0.06085567, -0.00219049, -0.0147152 ]],

        [[-0.08785438, -0.11267045,  0.07574052],
         [-0.03042783, -0.00109525, -0.0147152 ]]]])

In [11]:
evaluate_sampler(
    OpTreeNodeList([circuit, circuit2]),
    OpTreeNodeList([test, op_clean, op_clean2]),
    param1,
    opdic1,
    Sampler(),
    dictionaries_combined=False,
)

pre-processing 0.002001047134399414
Total number of circuits: 6
sampler run time 0.08499956130981445
circuit_operator_list[index_slice] [[[0], [1], [2, 3]], [[0], [1], [2, 3]]]
expec [[ 0.02610168 -0.29335694 -0.06118615  0.06180155]
 [ 0.09946798 -0.35737979  0.0742432  -0.00303811]]
expec2 [array([-0.26725525, -0.06118615,  0.06180155]), array([-0.25791181,  0.0742432 , -0.00303811])]
post processing 0.0030007362365722656


array([[-0.26725525, -0.06118615,  0.06180155],
       [-0.25791181,  0.0742432 , -0.00303811]])

In [12]:
evaluate_sampler(
    OpTreeNodeList([circuit, circuit2]),
    OpTreeNodeList([test, op_clean, op_clean2]),
    param2,
    opdic2,
    Sampler(),
    dictionaries_combined=False,
)

pre-processing 0.0019991397857666016
Total number of circuits: 6
sampler run time 0.024000167846679688
circuit_operator_list[index_slice] [[[0], [1], [2, 3]], [[0], [1], [2, 3]]]
expec [[ 0.05882409 -0.14667847 -0.11267045  0.07574052]
 [ 0.04973399 -0.08016182 -0.00109525 -0.0147152 ]]
expec2 [array([-0.08785438, -0.11267045,  0.07574052]), array([-0.03042783, -0.00109525, -0.0147152 ])]
post processing 0.0020012855529785156


array([[-0.08785438, -0.11267045,  0.07574052],
       [-0.03042783, -0.00109525, -0.0147152 ]])

In [13]:
evaluate_sampler(
    OpTreeNodeList([circuit, circuit2]),
    OpTreeNodeList([test, op_clean, op_clean2]),
    [param1, param2],
    [opdic1, opdic2],
    Sampler(),
    dictionaries_combined=False,
)

pre-processing 0.0050008296966552734
Total number of circuits: 12
sampler run time 0.02899909019470215
circuit_operator_list[index_slice] [[[0], [1], [2, 3]], [[0], [1], [2, 3]], [[0], [1], [2, 3]], [[0], [1], [2, 3]]]
expec [[ 0.02610168 -0.29335694 -0.06118615  0.06180155]
 [ 0.09946798 -0.35737979  0.0742432  -0.00303811]
 [ 0.11764818 -0.29335694 -0.22534089  0.07574052]
 [ 0.09946798 -0.16032365 -0.00219049 -0.0147152 ]]
expec2 [array([-0.26725525, -0.06118615,  0.06180155]), array([-0.25791181,  0.0742432 , -0.00303811]), array([-0.17570876, -0.22534089,  0.07574052]), array([-0.06085567, -0.00219049, -0.0147152 ])]
circuit_operator_list[index_slice] [[[0], [1], [2, 3]], [[0], [1], [2, 3]], [[0], [1], [2, 3]], [[0], [1], [2, 3]]]
expec [[ 0.01305084 -0.14667847 -0.03059308  0.06180155]
 [ 0.04973399 -0.17868989  0.0371216  -0.00303811]
 [ 0.05882409 -0.14667847 -0.11267045  0.07574052]
 [ 0.04973399 -0.08016182 -0.00109525 -0.0147152 ]]
expec2 [array([-0.13362763, -0.03059308,  0

array([[[[-0.26725525, -0.06118615,  0.06180155],
         [-0.25791181,  0.0742432 , -0.00303811]],

        [[-0.13362763, -0.03059308,  0.06180155],
         [-0.1289559 ,  0.0371216 , -0.00303811]]],


       [[[-0.17570876, -0.22534089,  0.07574052],
         [-0.06085567, -0.00219049, -0.0147152 ]],

        [[-0.08785438, -0.11267045,  0.07574052],
         [-0.03042783, -0.00109525, -0.0147152 ]]]])

In [14]:
expec_tree = gen_expectation_tree(
    OpTreeNodeList([circuit, circuit2]), OpTreeNodeList([test, op_clean, op_clean2])
)

full_dict = param1.copy()
full_dict.update(opdic1)

evaluate_expectation_tree_from_estimator(expec_tree, [full_dict, full_dict], Estimator())

Pre-processing:  0.010999441146850586
Number of circuits in the estimator: 16
Run time of estimator: 0.021997928619384766
Post-processing:  0.0


array([[[-0.26725525, -0.06118615,  0.06180155],
        [-0.25791181,  0.0742432 , -0.00303811]],

       [[-0.26725525, -0.06118615,  0.06180155],
        [-0.25791181,  0.0742432 , -0.00303811]]])

In [15]:
evaluate_expectation_tree_from_sampler(expec_tree, [full_dict, full_dict], Sampler())

Number of circuits in sampler call: 12
run time 0.039003849029541016
Post-processing 0.0019991397857666016


array([[[-0.26725525, -0.06118615,  0.06180155],
        [-0.25791181,  0.0742432 , -0.00303811]],

       [[-0.26725525, -0.06118615,  0.06180155],
        [-0.25791181,  0.0742432 , -0.00303811]]])

In [16]:
evaluate_expectation_tree_from_sampler(expec_tree, full_dict, Sampler())

Number of circuits in sampler call: 6
run time 0.01999974250793457
Post-processing 0.0010018348693847656


array([[-0.26725525, -0.06118615,  0.06180155],
       [-0.25791181,  0.0742432 , -0.00303811]])

In [17]:
# evaluate_expectation_tree_from_estimator(tree,{},Estimator())

In [18]:
# # Create a backend estimator
# backend = Aer.get_backend('statevector_simulator')
# estimator = BackendEstimator(backend=backend,options={'shots':1000000})

# job = estimator.run(circuit,op)

In [19]:
# job.result().values